In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sqlalchemy import create_engine
from sklearn2pmml.decoration import DateTimeDomain
%matplotlib inline

In [2]:
stock_df = pd.DataFrame()
symbols = []
import os
# for dirname, _, filenames in os.walk('/kaggle/input/stock-market-dataset/stocks'):
#     for filename in filenames:
#         symbol = filename.split('.')[0]
#         symbols.append(symbol)
#         new_df = pd.read_csv(os.path.join(dirname, filename))
#         new_df['Symbol'] = symbol
#         stock_df = pd.concat([stock_df,new_df], ignore_index = True)

for dirname, _, filenames in os.walk(r'C:\Users\Aidan\Documents\StocksAnalysis\stocks\stocks'):
    for i in range(1,100):
        symbol = filenames[i].split('.')[0]
        symbols.append(symbol)
        new_df = pd.read_csv(os.path.join(dirname, filenames[i]))
        new_df['Symbol'] = symbol
        stock_df = pd.concat([stock_df,new_df], ignore_index = True)

In [3]:
stock_df.shape

(442510, 8)

In [4]:
stock_df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,4.425090e+05,4.425090e+05,4.425090e+05,4.425090e+05,4.425090e+05,4.425090e+05
mean,2.883437e+03,2.998530e+03,2.755826e+03,2.867851e+03,-3.946707e+18,3.097755e+06
std,5.182033e+04,5.490669e+04,4.841652e+04,5.132398e+04,8.139392e+19,1.822903e+07
min,0.000000e+00,3.500000e-02,3.500000e-02,3.500000e-02,-2.380501e+21,0.000000e+00
25%,5.600000e+00,6.120000e+00,5.875000e+00,6.000000e+00,4.150000e+00,1.540000e+04
50%,1.515000e+01,1.550000e+01,1.504000e+01,1.529480e+01,1.150000e+01,1.897000e+05
75%,3.280000e+01,3.335000e+01,3.239000e+01,3.287000e+01,2.840000e+01,1.111800e+06
max,5.842935e+06,6.327720e+06,5.369490e+06,5.726700e+06,1.883516e+19,1.855410e+09


In [5]:
stock_df.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,1962-01-02,6.532155,6.556185,6.532155,6.532155,1.536658,55900.0,AA
1,1962-01-03,6.532155,6.632280,6.524145,6.632280,1.560212,74500.0,AA
2,1962-01-04,6.632280,6.664320,6.632280,6.632280,1.560212,80500.0,AA
3,1962-01-05,6.632280,6.656310,6.616260,6.624270,1.558326,70500.0,AA
4,1962-01-08,6.608250,6.608250,6.339915,6.408000,1.507450,93800.0,AA


In [6]:
stock_df.isnull().any()

Date         False
Open          True
High          True
Low           True
Close         True
Adj Close     True
Volume        True
Symbol       False
dtype: bool

In [7]:
stock_df = stock_df.fillna(lambda x: x.median())

In [8]:
stock_df.isnull().any()

Date         False
Open         False
High         False
Low          False
Close        False
Adj Close    False
Volume       False
Symbol       False
dtype: bool

In [9]:
stock_df['Symbol'].unique()

array(['AA', 'AACG', 'AAL', 'AAMC', 'AAME', 'AAN', 'AAOI', 'AAON', 'AAP',
       'AAPL', 'AAT', 'AAU', 'AAWW', 'AAXN', 'AB', 'ABB', 'ABBV', 'ABC',
       'ABCB', 'ABEO', 'ABEV', 'ABG', 'ABIO', 'ABM', 'ABMD', 'ABR', 'ABT',
       'ABTX', 'ABUS', 'AC', 'ACA', 'ACAD', 'ACAM', 'ACAMU', 'ACB',
       'ACBI', 'ACC', 'ACCO', 'ACEL', 'ACER', 'ACGL', 'ACGLO', 'ACGLP',
       'ACH', 'ACHC', 'ACHV', 'ACIA', 'ACIU', 'ACIW', 'ACLS', 'ACM',
       'ACMR', 'ACN', 'ACNB', 'ACOR', 'ACP', 'ACRE', 'ACRS', 'ACRX',
       'ACST', 'ACTG', 'ACTT', 'ACTTU', 'ACU', 'ACV', 'ACY', 'ADAP',
       'ADBE', 'ADC', 'ADES', 'ADI', 'ADIL', 'ADM', 'ADMA', 'ADMP',
       'ADMS', 'ADNT', 'ADP', 'ADPT', 'ADRO', 'ADS', 'ADSK', 'ADSW',
       'ADT', 'ADTN', 'ADUS', 'ADVM', 'ADX', 'ADXN', 'ADXS', 'AE', 'AEB',
       'AEE', 'AEF', 'AEFC', 'AEG', 'AEGN', 'AEHR', 'AEIS'], dtype=object)

In [10]:
stock_df = pd.get_dummies(stock_df, columns=["Symbol"], prefix=["Stock_is"] )

In [11]:
stock_df.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume,Stock_is_AA,Stock_is_AACG,Stock_is_AAL,...,Stock_is_ADXS,Stock_is_AE,Stock_is_AEB,Stock_is_AEE,Stock_is_AEF,Stock_is_AEFC,Stock_is_AEG,Stock_is_AEGN,Stock_is_AEHR,Stock_is_AEIS
0,1962-01-02,6.53216,6.55618,6.53216,6.53216,1.53666,55900,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1962-01-03,6.53216,6.63228,6.52415,6.63228,1.56021,74500,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1962-01-04,6.63228,6.66432,6.63228,6.63228,1.56021,80500,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1962-01-05,6.63228,6.65631,6.61626,6.62427,1.55833,70500,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1962-01-08,6.60825,6.60825,6.33991,6.408,1.50745,93800,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
X = stock_df.drop('Open', axis=1)
Y = stock_df['Open']

In [13]:
X.columns

Index(['Date', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Stock_is_AA',
       'Stock_is_AACG', 'Stock_is_AAL', 'Stock_is_AAMC',
       ...
       'Stock_is_ADXS', 'Stock_is_AE', 'Stock_is_AEB', 'Stock_is_AEE',
       'Stock_is_AEF', 'Stock_is_AEFC', 'Stock_is_AEG', 'Stock_is_AEGN',
       'Stock_is_AEHR', 'Stock_is_AEIS'],
      dtype='object', length=105)

In [14]:
X['Date'] = pd.to_datetime(X['Date'])

In [15]:
X['Date']=X['Date'].map(datetime.datetime.toordinal)

In [16]:
X.head(5)

,Date,High,Low,Close,Adj Close,Volume,Stock_is_AA,Stock_is_AACG,Stock_is_AAL,Stock_is_AAMC,...,Stock_is_ADXS,Stock_is_AE,Stock_is_AEB,Stock_is_AEE,Stock_is_AEF,Stock_is_AEFC,Stock_is_AEG,Stock_is_AEGN,Stock_is_AEHR,Stock_is_AEIS
0,716242,6.55618,6.53216,6.53216,1.53666,55900,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,716243,6.63228,6.52415,6.63228,1.56021,74500,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,716244,6.66432,6.63228,6.63228,1.56021,80500,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,716245,6.65631,6.61626,6.62427,1.55833,70500,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,716248,6.60825,6.33991,6.408,1.50745,93800,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [18]:
x_train.shape, x_test.shape

((354008, 105), (88502, 105))

In [19]:
y_train.shape, y_test.shape

((354008,), (88502,))

In [20]:
from sklearn.linear_model import LinearRegression

linear_model = LinearRegression(normalize=True, fit_intercept=False).fit(x_train, y_train)

TypeError: float() argument must be a string or a number, not 'function'